In [ ]:
# 필요한 모듈 임포트
import os
from dotenv import load_dotenv
import httpx
import time

class CustomHTTPClient(httpx.Client):
    def __init__(self, *args, **kwargs):
        kwargs.pop("proxies", None) # Remove the 'proxies' argument if present
        super().__init__(*args, **kwargs)

from openai import OpenAI

#환경변수 취득
load_dotenv()
#OpenAI API 클라이언트 생성
client = OpenAI(api_key=os.environ['API_KEY'], http_client=CustomHTTPClient())

# 모델명
# model_name = "gpt-3.5-turbo-0125"
# 2. 업로드할 학습 데이터 경로 (JSONL 형식)
file_path = "training.jsonl"  # 실제 경로로 변경

# 3. 모델명 (파인튜닝 지원 모델이어야 함)
model_name = "gpt-3.5-turbo-1106"

# 4. 파일 업로드
print("📁 파일 업로드 중...")
uploaded_file = client.files.create(
    file=open(file_path, "rb"),
    purpose="fine-tune"
)
print("✅ 업로드 완료! File ID:", uploaded_file.id)




📁 파일 업로드 중...
✅ 업로드 완료! File ID: file-19JhtnseqguFEWjd5SQaS1


In [ ]:
# 5. 파인튜닝 job 생성
print("🚀 파인튜닝 Job 생성 중...")
fine_tune_job = client.fine_tuning.jobs.create(
    training_file=uploaded_file.id,
    model=model_name
)
job_id = fine_tune_job.id
print("✅ Job 생성됨! Job ID:", job_id)

# 6. Job 상태 모니터링
print("⏳ 파인튜닝 진행 상황 확인 중...")
while True:
    job_status = client.fine_tuning.jobs.retrieve(job_id)
    status = job_status.status
    print(f"🔄 현재 상태: {status}")
    if status in ["succeeded", "failed", "cancelled"]:
        break
    time.sleep(10)  # 10초 간격으로 확인

# 7. 로그 출력 (선택)


🚀 파인튜닝 Job 생성 중...
✅ Job 생성됨! Job ID: ftjob-ZzpbCzIV8pfyYGzM9Yjs8duw
⏳ 파인튜닝 진행 상황 확인 중...
🔄 현재 상태: validating_files
🔄 현재 상태: validating_files
🔄 현재 상태: validating_files
🔄 현재 상태: validating_files
🔄 현재 상태: validating_files
🔄 현재 상태: validating_files
🔄 현재 상태: validating_files
🔄 현재 상태: running
🔄 현재 상태: running
🔄 현재 상태: running
🔄 현재 상태: running
🔄 현재 상태: running
🔄 현재 상태: running
🔄 현재 상태: running
🔄 현재 상태: running
🔄 현재 상태: running
🔄 현재 상태: running
🔄 현재 상태: running
🔄 현재 상태: running
🔄 현재 상태: running
🔄 현재 상태: running
🔄 현재 상태: running
🔄 현재 상태: running
🔄 현재 상태: running
🔄 현재 상태: running
🔄 현재 상태: running
🔄 현재 상태: running
🔄 현재 상태: running
🔄 현재 상태: running
🔄 현재 상태: running
🔄 현재 상태: running
🔄 현재 상태: running
🔄 현재 상태: running
🔄 현재 상태: running
🔄 현재 상태: running
🔄 현재 상태: succeeded

📜 이벤트 로그:


TypeError: Jobs.list_events() got an unexpected keyword argument 'id'

In [6]:
client.fine_tuning.jobs.retrieve(job_id)

FineTuningJob(id='ftjob-ZzpbCzIV8pfyYGzM9Yjs8duw', created_at=1744345483, error=Error(code=None, message=None, param=None), fine_tuned_model='ft:gpt-3.5-turbo-1106:personal::BL0KEEow', finished_at=1744345836, hyperparameters=Hyperparameters(batch_size=1, learning_rate_multiplier=2.0, n_epochs=10), model='gpt-3.5-turbo-1106', object='fine_tuning.job', organization_id='org-BEfrLC4bCyHxjR7Ri25802FM', result_files=['file-Y3Gr7ksntWkc7Hy7kjG4JJ'], seed=1360532911, status='succeeded', trained_tokens=2230, training_file='file-19JhtnseqguFEWjd5SQaS1', validation_file=None, estimated_finish=None, integrations=[], metadata=None, method=Method(dpo=None, supervised=MethodSupervised(hyperparameters=MethodSupervisedHyperparameters(batch_size=1, learning_rate_multiplier=2.0, n_epochs=10)), type='supervised'), user_provided_suffix=None)

In [9]:
# プロンプトの設定
content = "'폭삭 속았수다'의 의미에 대해 알고 싶어"

# APIへリクエスト
response = client.chat.completions.create(
  model="ft:gpt-3.5-turbo-1106:personal::BL0KEEow",
  messages=[
    {"role": "user", "content": content}
  ]
)

# 言語モデルからの回答を表示
print(response.choices[0].message.content.strip())

'폭삭 속았수다'는 한국어로 "정말 속았네요"라는 뜻입니다
